# L1: NLP Tasks With a Simple Interface

In [27]:
import os
import io
from IPython.display import Image, display, HTML
from PIL import Image
import base64 
from dotenv import load_dotenv, find_dotenv
import requests, json
load_dotenv(find_dotenv(), override=True)

import gradio as gr
import requests
import os

In [2]:
hf_api_key = os.environ.get("hf_api_key")
ENDPOINT_URL = os.environ.get("ENDPOINT_URL", os.environ.get("HF_API_SUMMARY_BASE"))

def get_completion(inputs, parameters=None):
    headers = {
        "Authorization": f"Bearer {hf_api_key}",
        "Content-Type": "application/json"
    }
    data = { "inputs": inputs }
    if parameters is not None:
        data.update({"parameters": parameters})
    response = requests.request("POST",
                                ENDPOINT_URL, headers=headers,
                                data=json.dumps(data)
                               )
    return json.loads(response.content.decode("utf-8"))

# Building a Text Summarization App

In [4]:
text = ('''The tower is 324 metres (1,063 ft) tall, about the same height
        as an 81-storey building, and the tallest structure in Paris. 
        Its base is square, measuring 125 metres (410 ft) on each side. 
        During its construction, the Eiffel Tower surpassed the Washington 
        Monument to become the tallest man-made structure in the world,
        a title it held for 41 years until the Chrysler Building
        in New York City was finished in 1930. It was the first structure 
        to reach a height of 300 metres. Due to the addition of a broadcasting 
        aerial at the top of the tower in 1957, it is now taller than the 
        Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the 
        Eiffel Tower is the second tallest free-standing structure in France 
        after the Millau Viaduct.''')

get_completion(text)

[{'summary_text': ' The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building . It is the tallest structure in Paris and the second tallest free-standing structure in France after the Millau Viaduct . It was the first structure in the world to reach a height of 300 metres .'}]

### Getting started with Gradio `gr.Interface` 

In [5]:
import gradio as gr
def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']
demo = gr.Interface(fn=summarize, inputs="text", outputs="text")
demo.launch()

c:\anaconda3\envs\deeplearning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


`demo.launch(share=True)` lets you create a public link to share with your team or friends.

In [6]:
def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']

gr.close_all()
demo = gr.Interface(fn=summarize, 
                    inputs=[gr.Textbox(label="Text to summarize", lines=6)],
                    outputs=[gr.Textbox(label="Result", lines=3)],
                    title="Text summarization with distilbart-cnn",
                    description="Summarize any text using the `shleifer/distilbart-cnn-12-6` model under the hood!"
                   )
demo.launch()

Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Building a Named Entity Recognition App

We are using this [Inference Endpoint](https://huggingface.co/inference-endpoints) for `dslim/bert-base-NER`, a 108M parameter fine-tuned BART model on the NER task.

#### gr.interface()
- Notice below that we pass in a list `[]` to `inputs` and to `outputs` because the function `fn` (in this case, `ner()`, can take in more than one input and return more than one output.
- The number of objects passed to `inputs` list should match the number of parameters that the `fn` function takes in, and the number of objects passed to the `outputs` list should match the number of objects returned by the `fn` function.

In [7]:
import gradio as gr
import requests

# Get the API URL and API key from environment variables
API_URL = os.environ.get("API_URL_NER")  
API_KEY = os.environ.get("API_KEY_NER")

# Function for NER
def ner(input):

    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {API_KEY}",  
        "Content-Type": "application/json"
    }

    payload = {
        "inputs": input,
        "parameters": {
            "aggregation_strategy": "simple"
        }
    }

    response = requests.post(API_URL, headers=headers, json=payload)
    output = response.json()

    return {"text": input, "entities": output}

gr.close_all() 
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with Wit.ai",
                    description="Find entities using the Wit.ai API under the hood!",
                    allow_flagging="never",
                    # Here we introduce a new tag, examples, easy to use examples for your application
                    examples=["My name is Sarah Jessica Parker but you can call me Jessica"])
demo.launch()

Closing server running on port: 7860
Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


### Adding a Helper Function to Merge Tokens

In [24]:
def merge_tokens(tokens):
    merged_tokens = []
    for token in tokens:
        if merged_tokens and token['entity'].startswith('I-') and merged_tokens[-1]['entity'].endswith(token['entity'][2:]):
            # If current token continues the entity of the last one, merge them
            last_token = merged_tokens[-1]
            last_token['word'] += token['word'].replace('##', '')
            last_token['end'] = token['end']
            last_token['score'] = (last_token['score'] + token['score']) / 2
        else:
            # Otherwise, add the token to the list
            merged_tokens.append(token)

    return merged_tokens

In [25]:
def ner(input):
    output = get_completion(input, parameters=None, API_URL=API_URL)
    merged_tokens = merge_tokens(output)
    return {"text": input, "entities": merged_tokens}

In [26]:
# Get the API URL and API key from environment variables
API_URL = os.environ.get("API_URL_NER")  
API_KEY = os.environ.get("API_KEY_NER")

# Function for NER
def ner(input):

    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {API_KEY}",  
        "Content-Type": "application/json"
    }

    payload = {
        "inputs": input,
        "parameters": {
            "aggregation_strategy": "simple"
        }
    }

    response = requests.post(API_URL, headers=headers, json=payload)
    output = response.json()

    return {"text": input, "entities": merge_tokens}

gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    examples=["My name is Andrew, I'm building DeeplearningAI and I live in California", "My name is Poli, I live in Vienna and work at HuggingFace"])

demo.launch()

Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [23]:
# Get the API URL and API key from environment variables
API_URL = os.environ.get("API_URL_NER")  
API_KEY = os.environ.get("API_KEY_NER")

# Function for NER
def ner(input):

    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {API_KEY}",  
        "Content-Type": "application/json"
    }

    payload = {
        "inputs": input,
        "parameters": {
            "aggregation_strategy": "simple"
        }
    }

    response = requests.post(API_URL, headers=headers, json=payload)
    output = response.json()

    return {"text": input, "entities": output}

gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    examples=[["My name is Andrew, I'm building DeeplearningAI and I live in California"],
                              ["My name is Poli, I live in Vienna and work at HuggingFace"]])

demo.launch()


Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
